In [ ]:
'''
Debugging Process for CSV File Reading Failures
說明：本筆記本記錄了CSV檔案讀取失敗的除錯流程
1. 識別並計算錯誤出現次數
2. 定位錯誤在檔案中的具體位置
3. 確定包含錯誤的行號
4. 根據上述資訊，精確定位錯誤位置，並返回HIS資料庫（藥局管理系統）進行資料內容的手動修正
5. 重新導出修正後的CSV檔案
6. 重新測試檔案讀取，驗證除錯是否成功

Description: This notebook documents the debugging process for CSV file reading failures.
1. Identify and count error occurrences
2. Locate the specific positions of errors within the file
3. Determine the line numbers containing errors
4. Based on the above information, precisely locate error positions and return to the HIS database (Pharmacy Management System) for manual data content correction
5. Re-export the corrected CSV file
6. Retest file reading to verify successful debugging
'''

In [ ]:
import pandas as pd

In [ ]:
path = "/content/2.藥物諮詢.csv"

In [ ]:
import pandas as pd
import io

# Try with 'utf-8' encoding
try:
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
    csv_data = pd.read_csv(io.StringIO(content))
    print("Successfully read with utf-8 encoding")
    print(csv_data.head())
except UnicodeDecodeError:
    print("Failed to read with utf-8 encoding")

# If utf-8 fails, try with 'cp950' (Traditional Chinese)
    try:
        with open(path, 'r', encoding='cp950') as file:
            content = file.read()
        csv_data = pd.read_csv(io.StringIO(content))
        print("Successfully read with cp950 encoding")
        print(csv_data.head())
    except UnicodeDecodeError:
        print("Failed to read with cp950 encoding")


Failed to read with utf-8 encoding
Failed to read with cp950 encoding


In [ ]:
import pandas as pd
import io

# Try with 'big5' encoding
try:
    with open(path, 'r', encoding='big5') as file:
        content = file.read()
    csv_data = pd.read_csv(io.StringIO(content))
    print("Successfully read with big5 encoding")
    print(csv_data.head())
except UnicodeDecodeError:
    print("Failed to read with big5 encoding")

# If big5 fails, try with 'big5hkscs' (Traditional Chinese)
    try:
        with open(path, 'r', encoding='big5hkscs') as file:
            content = file.read()
        csv_data = pd.read_csv(io.StringIO(content))
        print("Successfully read with big5hkscs encoding")
        print(csv_data.head())
    except UnicodeDecodeError:
        print("Failed to read with big5hkscs encoding")


Failed to read with big5 encoding
Failed to read with big5hkscs encoding


In [ ]:
import pandas as pd
import io

# Read file in binary mode
with open(path, 'rb') as file:
    content = file.read()

# Try different encodings
encodings_to_try = ['utf-8', 'big5', 'gb18030', 'cp950', 'shift_jis', 'big5hkscs']

for enc in encodings_to_try:
    try:
        decoded_content = content.decode(enc)
        csv_data = pd.read_csv(io.StringIO(decoded_content))
        print(f"Successfully read with {enc} encoding")
        print(csv_data.head())
        break
    except (UnicodeDecodeError, pd.errors.ParserError):
        print(f"Failed to read with {enc} encoding")


Failed to read with utf-8 encoding
Failed to read with big5 encoding
Failed to read with gb18030 encoding
Failed to read with cp950 encoding
Failed to read with shift_jis encoding
Failed to read with big5hkscs encoding


In [ ]:
import pandas as pd
import io

# Read file with error handling
with open(path, 'r', encoding='big5hkscs', errors='replace') as file:
    content = file.read()
    replacement_char = '\ufffd'  # Unicode replacement character
    num_replacements = content.count(replacement_char)
    print(f"Number of replaced characters: {num_replacements}")


csv_data = pd.read_csv(io.StringIO(content))
print("Read file with error handling")
print(csv_data.head())

with open(path, 'rb') as file:
    original_content = file.read()

print(f"Original file length: {len(original_content)} bytes")
print(f"Decoded content length: {len(content.encode('utf-8'))} bytes")


Number of replaced characters: 7
Read file with error handling
    藥品代碼                     藥品學名                       藥品名稱  藥品分類  \
0   OEZE                Ezetimibe               Ezetrol 10mg  CAVS   
1  OELME        Dihydroergotoxine               Elmesatt 2mg  CAVS   
2  OCHA1              Varenicline                CHAMPIX 1mg  ZADT   
3   EAZE               Azelastine  Azela nasal spray 100dose  TENT   
4   EPAT  NaCl + KCl + Boric Acid     Patear eye lotion 10mL  TOPH   

                                   適應症  \
0                                  NaN   
1                                  NaN   
2  Aid to smoking cessation treatment.   
3                                  NaN   
4         Dry eyes， eye wash solution.   

                                                  禁忌  \
0                              適應症:為降血脂藥(B024058100)   
1  適應症:Adjunct in treating symptoms of mild to mo...   
2                                                NaN   
3  適應症: Allergic rhinitis. 副作用: Rarely，

In [ ]:
with open(path, 'rb') as file:
    original_content = file.read()

print(f"Original file length: {len(original_content)} bytes")
print(f"Decoded content length: {len(content.encode('utf-8'))} bytes")


Original file length: 3230299 bytes
Decoded content length: 3523692 bytes


In [ ]:
positions = [i for i, char in enumerate(content) if char == '\ufffd']
print(f"Replaced character positions: {positions}")


Replaced character positions: [2340027, 2340056, 2340089, 2340183, 2340233, 2340288, 2340383]


In [ ]:
context_size = 20  # number of characters to show before and after
for pos in positions[:10]:  # limit to first 10 occurrences
    start = max(0, pos - context_size)
    end = min(len(content), pos + context_size + 1)
    print(f"Replacement at position {pos}:")
    print(content[start:end])
    print("-" * 40)


Replacement at position 2340027:
t diseases such as: �h unstable angina pe
----------------------------------------
Replacement at position 2340056:
le angina pectoris， �h recent myocardial 
----------------------------------------
Replacement at position 2340089:
cardial infarction， �h refractory arrhyth
----------------------------------------
Replacement at position 2340183:
tinuous arrhythmia， �h untreated or uncon
----------------------------------------
Replacement at position 2340233:
evere hypertension， �h untreated or uncon
----------------------------------------
Replacement at position 2340288:
tive heart failure， �{ Concomitant treatm
----------------------------------------
Replacement at position 2340383:
ic antidepressants. �{ Xylestesin-A is no
----------------------------------------


In [ ]:
nan_counts = csv_data.isna().sum()
print("NaN counts per column:")
print(nan_counts)


NaN counts per column:
藥品代碼             0
藥品學名            23
藥品名稱             1
藥品分類             0
適應症           1408
禁忌             323
副作用           1434
儲存             541
用法用量           264
肝功能異常(Y/N)    1170
肝功能異常說明       2172
腎功能異常(Y/N)    1181
懷孕用藥危險分級       373
孕期用藥建議        1333
孕期附帶說明        2012
哺乳期用藥建議        774
哺乳期附帶說明       1948
給藥途徑          1281
乾粉稀釋液         2624
輸注點滴液         2502
IVP用法建議       2588
IVD用法建議       2484
注意事項          1922
dtype: int64


In [ ]:
rows_with_replacements = csv_data.apply(lambda row: row.astype(str).str.contains('\ufffd').any(), axis=1)
print(f"Number of rows with replacements: {rows_with_replacements.sum()}")
print("Indices of rows with replacements:")
print(rows_with_replacements[rows_with_replacements].index.tolist())


Number of rows with replacements: 1
Indices of rows with replacements:
[2334]
